In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 확인

In [2]:
# library import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# data 확인
# train_path = 'data/train_dataset.csv'
# test_path = 'data/test_dataset.csv'
train_path = '/content/drive/MyDrive/에이블스쿨/실습파일/2023.04.12_미니프로젝트5차_3_5일차 실습자료/Kaggle_Competition/data/train_dataset.csv'
test_path = '/content/drive/MyDrive/에이블스쿨/실습파일/2023.04.12_미니프로젝트5차_3_5일차 실습자료/Kaggle_Competition/data/test_dataset.csv'

train_data = pd.read_csv(train_path)
x_test = pd.read_csv(test_path)

train_data.shape, x_test.shape

((5881, 52), (1471, 50))

- Dynamic : 0, 1, 2 (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS)
- Static : 3, 4, 5 (LAYING, STANDING, SITTING)

In [4]:
from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler()
temp_x = train_data.drop(['Activity', 'Activity_dynamic'], axis=1)
temp_x = mm_scaler.fit_transform(temp_x)

temp_x = pd.DataFrame(temp_x, columns=[list(train_data)[:-2]])
temp_y = train_data[['Activity', 'Activity_dynamic']]

train_data = pd.concat([temp_x, temp_y], axis=1, join='outer')

# 모델링

In [5]:
import tensorflow as tf
import numpy as np
import random
import os

def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # np
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    tf.random.set_seed(seed) # tensorflow

my_seed = 42
my_seed_everywhere(my_seed)

## PyCaret 사용해보기

In [6]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting req

In [7]:
temp_data = train_data.drop('Activity', axis=1)

from pycaret.classification import ClassificationExperiment
s = ClassificationExperiment()
s.setup(temp_data, target = 'Activity_dynamic', session_id = 42)

,Description,Value
0,Session id,42
1,Target,Activity_dynamic
2,Target type,Multiclass
3,Original data shape,"(5881, 51)"
4,Transformed data shape,"(5881, 51)"
5,Transformed train set shape,"(4116, 51)"
6,Transformed test set shape,"(1765, 51)"
7,Numeric features,50
8,Preprocess,True
9,Imputation type,simple


In [8]:
best = s.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7476,0.9549,0.7476,0.7480,0.7463,0.6961,0.6966,4.0070
xgboost,Extreme Gradient Boosting,0.7442,0.9537,0.7442,0.7441,0.7425,0.6920,0.6926,17.5790
gbc,Gradient Boosting Classifier,0.7170,0.9465,0.7170,0.7166,0.7152,0.6593,0.6599,33.9730
et,Extra Trees Classifier,0.7024,0.9425,0.7024,0.7014,0.6982,0.6415,0.6430,1.3150
rf,Random Forest Classifier,0.6985,0.9394,0.6985,0.6972,0.6955,0.6369,0.6379,2.4420
lda,Linear Discriminant Analysis,0.6628,0.9280,0.6628,0.6599,0.6586,0.5939,0.5949,0.1210
lr,Logistic Regression,0.6494,0.9237,0.6494,0.6421,0.6407,0.5776,0.5799,0.8360
ridge,Ridge Classifier,0.6458,0.0000,0.6458,0.6366,0.6222,0.5729,0.5801,0.0690
svm,SVM - Linear Kernel,0.6455,0.0000,0.6455,0.6582,0.6182,0.5724,0.5854,0.2240
knn,K Neighbors Classifier,0.5870,0.8809,0.5870,0.5934,0.5777,0.5018,0.5051,0.1150


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [9]:
print(best)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


# 동적 정적 구분 후 세부 분류

## is_dynamic 분류

In [10]:
train_data['is_dynamic'] = np.where(train_data['Activity_dynamic'] < 3, 1, 0)

In [11]:
train_data.groupby(['Activity', 'is_dynamic']).count().index

MultiIndex([(            'LAYING', 0),
            (           'SITTING', 0),
            (          'STANDING', 0),
            (           'WALKING', 1),
            ('WALKING_DOWNSTAIRS', 1),
            (  'WALKING_UPSTAIRS', 1)],
           names=['Activity', 'is_dynamic'])

In [12]:
temp_data = train_data.drop(['Activity', 'Activity_dynamic'], axis=1)

from pycaret.classification import ClassificationExperiment
s = ClassificationExperiment()
s.setup(temp_data, target = 'is_dynamic', session_id = 42)

,Description,Value
0,Session id,42
1,Target,is_dynamic
2,Target type,Binary
3,Original data shape,"(5881, 51)"
4,Transformed data shape,"(5881, 51)"
5,Transformed train set shape,"(4116, 51)"
6,Transformed test set shape,"(1765, 51)"
7,Numeric features,50
8,Preprocess,True
9,Imputation type,simple


In [13]:
best = s.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9993,1.0000,1.0000,0.9984,0.9992,0.9985,0.9985,0.7730
ada,Ada Boost Classifier,0.9990,1.0000,0.9995,0.9984,0.9989,0.9980,0.9980,1.4540
rf,Random Forest Classifier,0.9988,1.0000,0.9989,0.9984,0.9987,0.9975,0.9976,1.1080
lr,Logistic Regression,0.9985,0.9999,0.9995,0.9973,0.9984,0.9971,0.9971,0.1050
lightgbm,Light Gradient Boosting Machine,0.9985,1.0000,0.9978,0.9989,0.9984,0.9971,0.9971,0.9230
knn,K Neighbors Classifier,0.9983,0.9998,1.0000,0.9962,0.9981,0.9966,0.9966,0.1200
gbc,Gradient Boosting Classifier,0.9983,0.9990,0.9984,0.9978,0.9981,0.9966,0.9966,4.7740
xgboost,Extreme Gradient Boosting,0.9983,1.0000,0.9984,0.9979,0.9981,0.9966,0.9966,0.7020
dt,Decision Tree Classifier,0.9981,0.9981,0.9984,0.9973,0.9978,0.9961,0.9961,0.1260
ridge,Ridge Classifier,0.9976,0.0000,1.0000,0.9947,0.9973,0.9951,0.9951,0.1320


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [14]:
print(best)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=-1, oob_score=False,
                     random_state=42, verbose=0, warm_start=False)


## 정적 Class 분류

In [15]:
static_data = train_data[train_data['is_dynamic'] == 0].reset_index(drop=True)
static_data['Activity_dynamic'] = static_data['Activity_dynamic'] - 3
static_data = static_data.drop(['Activity', 'is_dynamic'], axis=1)

In [16]:
from pycaret.classification import ClassificationExperiment
s = ClassificationExperiment()
s.setup(static_data, target = 'Activity_dynamic', session_id = 42)

,Description,Value
0,Session id,42
1,Target,Activity_dynamic
2,Target type,Multiclass
3,Original data shape,"(3234, 51)"
4,Transformed data shape,"(3234, 51)"
5,Transformed train set shape,"(2263, 51)"
6,Transformed test set shape,"(971, 51)"
7,Numeric features,50
8,Preprocess,True
9,Imputation type,simple


In [17]:
best = s.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6889,0.8530,0.6889,0.6865,0.6858,0.5326,0.5341,2.1350
xgboost,Extreme Gradient Boosting,0.6823,0.8510,0.6823,0.6797,0.6789,0.5227,0.5243,6.5790
gbc,Gradient Boosting Classifier,0.6580,0.8354,0.6580,0.6547,0.6532,0.4859,0.4882,8.9720
et,Extra Trees Classifier,0.6438,0.8217,0.6438,0.6389,0.6341,0.4639,0.4692,0.8850
rf,Random Forest Classifier,0.6408,0.8169,0.6408,0.6356,0.6322,0.4595,0.4639,1.5580
lda,Linear Discriminant Analysis,0.6235,0.7953,0.6235,0.6183,0.6186,0.4343,0.4359,0.2220
ada,Ada Boost Classifier,0.6067,0.7811,0.6067,0.6106,0.6077,0.4100,0.4107,1.0570
ridge,Ridge Classifier,0.5970,0.0000,0.5970,0.5802,0.5704,0.3920,0.4036,0.0820
lr,Logistic Regression,0.5766,0.7654,0.5766,0.5620,0.5601,0.3622,0.3682,0.2540
svm,SVM - Linear Kernel,0.5484,0.0000,0.5484,0.5811,0.4868,0.3178,0.3587,0.1450


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [18]:
print(best)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


## 동적 Class 분류

In [19]:
dynamic_data = train_data[train_data['is_dynamic'] == 1].reset_index(drop=True)
dynamic_data = dynamic_data.drop(['Activity', 'is_dynamic'], axis=1)

from pycaret.classification import ClassificationExperiment
s = ClassificationExperiment()
s.setup(dynamic_data, target = 'Activity_dynamic', session_id = 42)

,Description,Value
0,Session id,42
1,Target,Activity_dynamic
2,Target type,Multiclass
3,Original data shape,"(2647, 51)"
4,Transformed data shape,"(2647, 51)"
5,Transformed train set shape,"(1852, 51)"
6,Transformed test set shape,"(795, 51)"
7,Numeric features,50
8,Preprocess,True
9,Imputation type,simple


In [20]:
best = s.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8385,0.9532,0.8385,0.8408,0.8381,0.7562,0.7576,1.8900
xgboost,Extreme Gradient Boosting,0.8180,0.9491,0.8180,0.8196,0.8176,0.7252,0.7263,4.8150
gbc,Gradient Boosting Classifier,0.7975,0.9345,0.7975,0.8005,0.7972,0.6943,0.6960,8.0510
lda,Linear Discriminant Analysis,0.7964,0.9307,0.7964,0.7992,0.7965,0.6931,0.6943,0.2270
ridge,Ridge Classifier,0.7808,0.0000,0.7808,0.7824,0.7800,0.6688,0.6703,0.0960
et,Extra Trees Classifier,0.7781,0.9242,0.7781,0.7798,0.7776,0.6654,0.6665,0.8560
rf,Random Forest Classifier,0.7743,0.9157,0.7743,0.7754,0.7736,0.6592,0.6603,1.3790
svm,SVM - Linear Kernel,0.7678,0.0000,0.7678,0.7813,0.7638,0.6498,0.6587,0.2200
qda,Quadratic Discriminant Analysis,0.7630,0.9041,0.7630,0.7781,0.7638,0.6439,0.6500,0.1190
lr,Logistic Regression,0.7619,0.9024,0.7619,0.7651,0.7615,0.6402,0.6419,0.2530


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [21]:
print(best)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
